In [25]:
import gymnasium as gym
import numpy as np

In [ ]:
NUM_EPISODES = 1000
LEARNING_RATE = 0.1
DISCOUNT_FACTOR = 0.9
EPSILON = 0.1

In [32]:
env = gym.make("ALE/Breakout-v5")
state_size = env.observation_space.shape
action_size = env.action_space.n
q_values = np.zeros((state_size[0], state_size[1], action_size))

In [27]:
color = np.array([210, 164, 74]).mean()

def preprocess_state(state):

    #crop and resize the image
    image = state[1:176:2, ::2]

    #convert the image to greyscale
    image = image.mean(axis=2)

    #improve image contrast
    image[image==color] = 0

    #normalize the image
    image = (image - 128) / 128 - 1
    
    #reshape the image
    image = np.expand_dims(image.reshape(88, 80, 1), axis=0)

    return image